# Data Processing

In [5]:
! unzip /content/InferredBugs-main.zip

Streaming output truncated to the last 5000 lines.
  inflating: InferredBugs-main/inferredbugs/java/super-csv/11/commit_info.json  
  inflating: InferredBugs-main/inferredbugs/java/super-csv/11/file_after.txt  
  inflating: InferredBugs-main/inferredbugs/java/super-csv/11/file_before.txt  
  inflating: InferredBugs-main/inferredbugs/java/super-csv/11/method_after.txt  
  inflating: InferredBugs-main/inferredbugs/java/super-csv/11/method_before.txt  
   creating: InferredBugs-main/inferredbugs/java/super-csv/2/
  inflating: InferredBugs-main/inferredbugs/java/super-csv/2/bug.json  
  inflating: InferredBugs-main/inferredbugs/java/super-csv/2/commit_info.json  
  inflating: InferredBugs-main/inferredbugs/java/super-csv/2/file_after.txt  
  inflating: InferredBugs-main/inferredbugs/java/super-csv/2/file_before.txt  
  inflating: InferredBugs-main/inferredbugs/java/super-csv/2/method_after.txt  
  inflating: InferredBugs-main/inferredbugs/java/super-csv/2/method_before.txt  
   creating: I

In [2]:
!pip install transformers datasets accelerate sentencepiece


In [6]:
import os
import pandas as pd

base_dir = "/content/InferredBugs-main/inferredbugs/csharp"  # Root directory
data = []

for project in os.listdir(base_dir):  # Loop each project folder
    project_path = os.path.join(base_dir, project)

    if os.path.isdir(project_path):  # Ensure it's a folder
        for bug_folder in os.listdir(project_path):  # Loop bug folders like 1,2,3…
            bug_path = os.path.join(project_path, bug_folder)

            if os.path.isdir(bug_path):
                before_file = os.path.join(bug_path, "method_before.txt")
                after_file = os.path.join(bug_path, "method_after.txt")

                if os.path.exists(before_file) and os.path.exists(after_file):
                    try:
                        with open(before_file, 'r', encoding='utf-8') as f:
                            buggy_code = f.read()
                        with open(after_file, 'r', encoding='utf-8') as f:
                            fixed_code = f.read()

                        data.append({
                            "input": buggy_code,
                            "target": fixed_code,
                            "project": project,
                            "bug_folder": bug_folder
                        })
                    except Exception as e:
                        print(f" Error reading {bug_path}: {e}")

df = pd.DataFrame(data)
print(f"📊 Total Bug Fix Samples Loaded: {len(df)}")
df.head()


📊 Total Bug Fix Samples Loaded: 2891


,input,target,project,bug_folder
0,[TestInitialize]\n public void ...,[TestInitialize]\n public void ...,abot,1
1,[Test]\n public void AccountAct...,[Test]\n public void AccountAct...,netmq,11
2,static void Main(string[] args)\n\t\t{\n\t\t...,static void Main(string[] args)\n\t\t{\n\t\t...,netmq,3
3,"public static int Poll(PollItem[] items, int...","public static int Poll(PollItem[] items, int...",netmq,5
4,"public static int Poll(PollItem[] items, int...","public static int Poll(PollItem[] items, int...",netmq,7


In [7]:
print(df["project"].value_counts())


project
cscs                      108
apm-agent-dotnet           97
resin                      55
couchbase-net-client       52
pact-net                   49
                         ... 
lex.db                      1
grpc-dotnet-namedpipes      1
LocalAppVeyor               1
PeanutButter                1
PTView                      1
Name: count, Length: 641, dtype: int64


In [8]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(df, test_size=0.1, random_state=42)

from datasets import Dataset
train_ds = Dataset.from_pandas(train_data)
test_ds  = Dataset.from_pandas(test_data)

In [9]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "Salesforce/codet5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/242M [00:00<?, ?B/s]

In [10]:
train_ds.column_names


['input', 'target', 'project', 'bug_folder', '__index_level_0__']

In [11]:
from sklearn.model_selection import train_test_split
from datasets import Dataset

# 1️⃣ Split pandas DataFrame
train_df, test_df = train_test_split(df, test_size=0.1, random_state=42)

# 2️⃣ Convert to HuggingFace Datasets
train_ds = Dataset.from_pandas(train_df)
test_ds = Dataset.from_pandas(test_df)

# 3️⃣ Tokenization Function (same as before)
def tokenize_function(example):
    model_inputs = tokenizer(
        example["input"], max_length=512, truncation=True
    )
    labels = tokenizer(
        example["target"], max_length=512, truncation=True
    )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# 4️⃣ Apply tokenization to both train and test
train_ds_tokenized = train_ds.map(tokenize_function, batched=True)
test_ds_tokenized = test_ds.map(tokenize_function, batched=True)



Map:   0%|          | 0/2601 [00:00<?, ? examples/s]

Map:   0%|          | 0/290 [00:00<?, ? examples/s]

In [32]:
pip install evaluate

In [35]:
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

training_args = TrainingArguments(
    output_dir="./bugfixer",
    eval_strategy="no",
    logging_strategy="steps",
    logging_steps=10,
    per_device_train_batch_size=2,
    num_train_epochs=3,
    save_strategy="epoch",
    report_to="none",
    fp16=True,          # Optional but improves speed on GPU
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds_tokenized,
    eval_dataset=test_ds_tokenized ,
    data_collator=data_collator,   # 👈 FIXED HERE
)

trainer.train()

Step,Training Loss
10,2.628600
20,1.173200
30,0.893700
40,0.884700
50,0.734600
60,0.530100
70,0.717700
80,0.851200
90,0.667000
100,0.421500


TrainOutput(global_step=3903, training_loss=0.3152099924151054, metrics={'train_runtime': 1411.1637, 'train_samples_per_second': 5.529, 'train_steps_per_second': 2.766, 'total_flos': 3103700181857280.0, 'train_loss': 0.3152099924151054, 'epoch': 3.0})

In [37]:
model.save_pretrained("/content/bugfixer_model")
tokenizer.save_pretrained("/content/bugfixer_model")


('/content/bugfixer_model/tokenizer_config.json',
 '/content/bugfixer_model/special_tokens_map.json',
 '/content/bugfixer_model/vocab.json',
 '/content/bugfixer_model/merges.txt',
 '/content/bugfixer_model/added_tokens.json',
 '/content/bugfixer_model/tokenizer.json')

In [38]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_path = "/content/bugfixer_model"

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)


In [39]:
def fix_buggy_code(buggy_code):
    input_text = f"Fix buggy code:\n{buggy_code}"

    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, padding=True)

    outputs = model.generate(
        **inputs,
        max_length=512,
        num_beams=4,
        early_stopping=True
    )

    return tokenizer.decode(outputs[0], skip_special_tokens=True)


In [48]:
buggy_code = """
public int Multiply(int a, int b)
{
    return a + b;  // BUG: addition instead of multiplication
}

"""

inputs = tokenizer(buggy_code, return_tensors="pt", truncation=True, max_length=512)

outputs = model.generate(
    inputs["input_ids"],
    max_length=512,
    num_beams=5,
    early_stopping=True
)

fixed_code = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(fixed_code)


public int Multiply(int a, int b)
{
    return a * b;

    }


In [53]:
buggy_code="""public int Additon(int a, int b)
{
    return a * b;  // BUG: multiplication instead of addition
}"""
inputs = tokenizer(buggy_code, return_tensors="pt", truncation=True, max_length=512)

outputs = model.generate(
    inputs["input_ids"],
    max_length=512,
    num_beams=5,
    early_stopping=True
)

fixed_code = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(fixed_code)

 public int Additon(int a, int b)
{
    return a + b;  // BUG: multiplication instead of addition 
}


In [54]:
buggy_code="""public int Subtraction(int a, int b)
{
    return a / b;  // BUG: divison instead of subtraction
}"""
inputs = tokenizer(buggy_code, return_tensors="pt", truncation=True, max_length=512)

outputs = model.generate(
    inputs["input_ids"],
    max_length=512,
    num_beams=5,
    early_stopping=True
)

fixed_code = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(fixed_code)

 public int Subtraction(int a, int b)
{
    return a - b;  // BUG: divison instead of subtraction
}


In [56]:
buggy_code="""public int Division(int a, int b)
{
    return a * b;
}"""
inputs = tokenizer(buggy_code, return_tensors="pt", truncation=True, max_length=512)

outputs = model.generate(
    inputs["input_ids"],
    max_length=512,
    num_beams=5,
    early_stopping=True
)

fixed_code = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(fixed_code)

 public int Division(int a, int b)
{
    return a / b;  
    }
